In [2]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import datetime as dt
import dateutil.parser
import re
import json
from pprint import pprint

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect

# EXTRACT

In [20]:
#Load in file scraped from ____ as text and converted to CSV
launchlog = pd.read_csv('Resources/launchlogall.csv')


#Load in files downloaded as CSV from kaggle.com
spacexdf = pd.read_csv('.\\Resources\\spaceX.csv')
nasadf = pd.read_csv('.\\Resources\\NASA_Facilities.csv')

#Load in file saved as JSON, retrieved from https://api.spacexdata.com/v2/launches
spacex = pd.read_json('.\\Resources\\spaceX.json')

In [ ]:
#Weather API call

# TRANSFORM

## Launch Log

In [5]:
launchlog.columns

Index(['Launch', 'Launch Date (UTC)', 'COSPAR', 'PL Name', 'Orig PL Name',
       'SATCAT', 'LV Type', 'LV S/N', 'Site', 'Suc', 'Ref'],
      dtype='object')

In [6]:
launchlog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8641 entries, 0 to 8640
Data columns (total 11 columns):
Launch               5464 non-null object
Launch Date (UTC)    5464 non-null object
COSPAR               8641 non-null object
PL Name              8641 non-null object
Orig PL Name         8641 non-null object
SATCAT               8641 non-null object
LV Type              5464 non-null object
LV S/N               5464 non-null object
Site                 5464 non-null object
Suc                  5464 non-null object
Ref                  5464 non-null object
dtypes: object(11)
memory usage: 742.7+ KB


In [7]:
launchlog = launchlog.rename(index=str, columns={"Launch": "launch", 
                                     "Launch Date (UTC)": "launch_date_utc",
                                     "COSPAR": "COSPAR",
                                     "PL Name": "pl_name",
                                     "Orig PL Name": "orig_pl_name",
                                     "SATCAT": "SATCAT",
                                     "LV Type": "lv_type",
                                     "LV S/N": "lv_sn",
                                     "Site": "site",
                                     "Suc": "suc"                                     
                                    })
launchlog.columns

Index(['launch', 'launch_date_utc', 'COSPAR', 'pl_name', 'orig_pl_name',
       'SATCAT', 'lv_type', 'lv_sn', 'site', 'suc', 'Ref'],
      dtype='object')

In [8]:
launchlog['launch_date_utc'] = launchlog['launch_date_utc'].fillna(method='ffill')
launchlog['launch'] = launchlog['launch'].fillna(method='ffill')

In [9]:
launchlog['launch_date_utc'] = launchlog['launch_date_utc'].fillna(method='ffill')
launchlog['launch_date'] = launchlog['launch_date_utc'].map(lambda x: dateutil.parser.parse(x[0:10]) if pd.notnull(x) else np.nan)\
                                                       .map(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else np.nan)

## Space X

In [21]:
#Parse JSON to get values
spacex['launch_date'] = spacex['launch_date_local'].map(lambda x: dateutil.parser.parse(x[0:10]) if pd.notnull(x) else np.nan)\
                                                       .map(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else np.nan)

spacex['launch_site'] = spacex['launch_site'].apply(lambda x: x.get('site_name_long'))

spacex['rocket_reused'] = spacex['rocket'].apply(lambda x: x.get('fairings') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('reused') if pd.notnull(x) else np.nan)

spacex['rocket_name'] = spacex['rocket'].apply(lambda x: x.get('rocket_name') if pd.notnull(x) else np.nan)

spacex['rocket_type'] = spacex['rocket'].apply(lambda x: x.get('rocket_type') if pd.notnull(x) else np.nan)

spacex['failure_reason'] = spacex['launch_failure_details'].apply(lambda x: x.get('reason') if pd.notnull(x) else np.nan)

spacex['customer'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('customers') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])

spacex['customer_country'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('nationality') if pd.notnull(x) else np.nan)


spacex['orbit'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('orbit') if pd.notnull(x) else np.nan)


spacex['payload_id'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('payload_id') if pd.notnull(x) else np.nan)

spacex['payload_mass_kg'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('payload_mass_kg') if pd.notnull(x) else np.nan)

spacex['payload_type'] = spacex['rocket'].apply(lambda x: x.get('second_stage') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x.get('payloads') if pd.notnull(x) else np.nan)\
                                            .apply(lambda x: x[0])\
                                            .apply(lambda x: x.get('payload_type') if pd.notnull(x) else np.nan)
# Drop unneeded columns
spacex.drop(['details', 'flight_number', 'is_tentative', 'launch_date_local','launch_date_unix', 'launch_date_utc',\
             'launch_failure_details','launch_window', 'launch_year','links', 'mission_id', 'reuse', 'rocket', 'ships',\
             'static_fire_date_unix', 'static_fire_date_utc', 'tbd', 'telemetry','tentative_max_precision', 'timeline',\
             'upcoming'],axis=1,inplace=True)

In [22]:
spacex.head()

,launch_site,launch_success,mission_name,launch_date,rocket_reused,rocket_name,rocket_type,failure_reason,customer,customer_country,orbit,payload_id,payload_mass_kg,payload_type
0,Kwajalein Atoll Omelek Island,False,FalconSat,2006-03-25,False,Falcon 1,Merlin A,merlin engine failure,DARPA,United States,LEO,FalconSAT-2,20.0,Satellite
1,Kwajalein Atoll Omelek Island,False,DemoSat,2007-03-21,False,Falcon 1,Merlin A,harmonic oscillation leading to premature engi...,DARPA,United States,LEO,DemoSAT,NaN,Satellite
2,Kwajalein Atoll Omelek Island,False,Trailblazer,2008-08-02,False,Falcon 1,Merlin C,residual stage-1 thrust led to collision betwe...,NASA,United States,LEO,Trailblazer,NaN,Satellite
3,Kwajalein Atoll Omelek Island,True,RatSat,2008-09-28,False,Falcon 1,Merlin C,NaN,SpaceX,United States,LEO,RatSat,165.0,Satellite
4,Kwajalein Atoll Omelek Island,True,RazakSat,2009-07-13,False,Falcon 1,Merlin C,NaN,ATSB,Malaysia,LEO,RazakSAT,200.0,Satellite


In [10]:
spacexdf.head(41)

,Flight Number,Launch Date,Launch Time,Launch Site,Vehicle Type,Payload Name,Payload Type,Payload Mass (kg),Payload Orbit,Customer Name,Customer Type,Customer Country,Mission Outcome,Failure Reason,Landing Type,Landing Outcome
0,F1-1,24 March 2006,22:30,Marshall Islands,Falcon 1,FalconSAT-2,Research Satellite,19.5,NaN,DARPA,Government,United States,Failure,Engine Fire During Launch,NaN,NaN
1,F1-2,21 March 2007,01:10,Marshall Islands,Falcon 1,DemoSat,NaN,NaN,NaN,DARPA,Government,United States,Failure,Engine Shutdown During Launch,NaN,NaN
2,F1-3,3 August 2008,03:34,Marshall Islands,Falcon 1,Trailblazer,Communication Satellite,NaN,NaN,ORS,Government,United States,Failure,Collision During Launch,NaN,NaN
3,F1-3,3 August 2008,03:34,Marshall Islands,Falcon 1,"PRESat, NanoSail-D",Research Satellites,8.0,NaN,NASA,Government,United States,Failure,Collision During Launch,NaN,NaN
4,F1-3,3 August 2008,03:34,Marshall Islands,Falcon 1,Explorers,Human Remains,NaN,NaN,Celestis,Business,United States,Failure,Collision During Launch,NaN,NaN
5,F1-4,28 September 2008,23:15,Marshall Islands,Falcon 1,RatSat (DemoSat),NaN,165.0,Low Earth Orbit,NaN,NaN,NaN,Success,NaN,NaN,NaN
6,F1-5,14 July 2009,03:35,Marshall Islands,Falcon 1,RazakSAT,Weather Satellite,180.0,Low Earth Orbit,ATSB,Government,Malaysia,Success,NaN,NaN,NaN
7,F9-1,4 June 2010,18:45,Cape Canaveral AFS LC-40,Falcon 9 (v1.0),Dragon Spacecraft Qualification Unit,NaN,NaN,Low Earth Orbit,NaN,NaN,NaN,Success,NaN,Parachute,Failure
8,F9-2,8 December 2010,15:43,Cape Canaveral AFS LC-40,Falcon 9 (v1.0),SpaceX CRS (Dragon C1),Space Station Supplies,NaN,Low Earth Orbit,NASA,Government,United States,Success,NaN,Parachute,Failure
9,F9-3,22 May 2012,07:44,Cape Canaveral AFS LC-40,Falcon 9 (v1.0),SpaceX CRS (Dragon C2+),Space Station Supplies,NaN,Low Earth Orbit,NASA,Government,United States,Success,NaN,NaN,NaN


In [11]:
#Format Dates to MM/DD/YYYY
spacexdf['Launch Date'] = spacexdf['Launch Date'].map(lambda x: dateutil.parser.parse(x))\
                                                    .map(lambda x: x.strftime('%Y-%m-%d'))

In [12]:
#rename columns prior to inserting into MYSQL
spacexdf = spacexdf.rename(index=str, columns={"Flight Number": "flight_number", 
                                            "Launch Date": "launch_date",
                                            "Launch Time": "launch_time",
                                            "Launch Site": "launch_site",
                                            "Vehicle Type": "vehicle_type",
                                            "Payload Name": "payload_name",
                                            "Payload Type": "payload_type",
                                            "Payload Mass (kg)": "payload_mass_kg",
                                            "Payload Orbit": "payload_orbit",
                                            "Customer Name": "customer_name",
                                            "Customer Type": "customer_type",
                                            "Customer Country": "customer_country",
                                            "Mission Outcome": "mission_outcome",
                                            "Failure Reason": "failure_reason",
                                            "Landing Type": "landing_type",
                                            "Landing Outcome": "landing_outcome" })

In [13]:
spacexdf.head()

,flight_number,launch_date,launch_time,launch_site,vehicle_type,payload_name,payload_type,payload_mass_kg,payload_orbit,customer_name,customer_type,customer_country,mission_outcome,failure_reason,landing_type,landing_outcome
0,F1-1,2006-03-24,22:30,Marshall Islands,Falcon 1,FalconSAT-2,Research Satellite,19.5,NaN,DARPA,Government,United States,Failure,Engine Fire During Launch,NaN,NaN
1,F1-2,2007-03-21,01:10,Marshall Islands,Falcon 1,DemoSat,NaN,NaN,NaN,DARPA,Government,United States,Failure,Engine Shutdown During Launch,NaN,NaN
2,F1-3,2008-08-03,03:34,Marshall Islands,Falcon 1,Trailblazer,Communication Satellite,NaN,NaN,ORS,Government,United States,Failure,Collision During Launch,NaN,NaN
3,F1-3,2008-08-03,03:34,Marshall Islands,Falcon 1,"PRESat, NanoSail-D",Research Satellites,8.0,NaN,NASA,Government,United States,Failure,Collision During Launch,NaN,NaN
4,F1-3,2008-08-03,03:34,Marshall Islands,Falcon 1,Explorers,Human Remains,NaN,NaN,Celestis,Business,United States,Failure,Collision During Launch,NaN,NaN


## Nasa Facilities

In [14]:
nasadf.head()

,Center,Center Search Status,Facility,FacilityURL,Occupied,Status,URL Link,Record Date,Last Update,Country,Location,City,State,Zipcode
0,Stennis Space Center,Public,Test Stand A-2 #4122,NaN,01/01/1964 12:00:00 AM,Inactive,http://rockettest.nasa.gov,03/01/1996 12:00:00 AM,04/06/2015 12:00:00 AM,US,"39529-6000 (30.385948, -89.604486)",Stennis Space Center,MS,39529-6000
1,Ames Research Center,Public,N229 - EXPER. AEROTHERMODYNAMIC FAC.: ELECTRIC...,NaN,01/01/1961 12:00:00 AM,Active,http://thermo-physics.arc.nasa.gov,03/01/1996 12:00:00 AM,06/13/2014 12:00:00 AM,US,"94035 (37.41412, -122.052585)",Moffett Field,CA,94035
2,Goddard Space Flight Center,Public,"500 WSC: Antenna, 10m S-band",NaN,01/01/1977 12:00:00 AM,Active,NaN,03/01/1996 12:00:00 AM,08/15/2013 12:00:00 AM,US,"20771 (38.99538, -76.853161)",Greenbelt,MD,20771
3,Glenn Research Center,Public,Vacuum Facility 11 (VF-11) Bldg. 16,NaN,NaN,Active,http://facilities.grc.nasa.gov/epl/index.html,07/15/2010 12:00:00 AM,03/04/2015 12:00:00 AM,US,"44135 (41.430364, -81.808561)",Cleveland,OH,44135
4,Marshall Space Flight Center,Public,Materials Testing & Research Facility 4464,NaN,NaN,Active,NaN,03/08/2007 12:00:00 AM,03/31/2014 12:00:00 AM,US,"35812 (34.729538, -86.585283)",Huntsville,AL,35812


In [15]:
#Extract Lat and Long from location column into separate columns
latlong = nasadf['Location'].str.strip('()')\
                   .str.split(', ', expand=True)\
                   .rename(columns={0:'Latitude', 1:'Longitude'})

#Remove Zip code and punctuation from location column
latlong['Latitude'] = latlong['Latitude'].map(lambda x: x[-9:])\
                        .str.replace('(','')

#Add lat/long to df
nasadf = pd.merge(nasadf, latlong, left_index=True, right_index=True)

#Remove time stamp from occupied, record date and last update columns
nasadf['Record Date'] = nasadf['Record Date'].map(lambda x: dateutil.parser.parse(x))\
                                                .map(lambda x: x.strftime('%Y-%m-%d'))

nasadf['Occupied'] = nasadf['Occupied'].map(lambda x: dateutil.parser.parse(x) if pd.notnull(x) else np.nan)\
                                        .map(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else np.nan)

nasadf['Last Update'] = nasadf['Last Update'].map(lambda x: dateutil.parser.parse(x)if pd.notnull(x) else np.nan)\
                                            .map(lambda x: x.strftime('%Y-%m-%d')if pd.notnull(x) else np.nan)
    
#Combine City and State into location column
nasadf['Location'] = nasadf['City'] + ', ' + nasadf['State']

# Drop unneeded columns
nasadf.drop(['FacilityURL'],axis=1,inplace=True)

In [16]:
#rename columns prior to inserting into MYSQL
nasadf = nasadf.rename(index=str, columns={"Center": "center", 
                                            "Center Search Status": "center_search_status",
                                            "Facility": "facility",
                                            "Occupied": "occupied",
                                            "Status": "status",
                                            "URL Link": "url",
                                            "Record Date": "record_date",
                                            "Last Update": "last_update",
                                            "Country": "country",
                                            "Location": "location",
                                            "City": "city",
                                            "State": "state",
                                            "Zipcode": "zipcode",
                                            "Latitude": "latitude",
                                            "Longitude": "longitude" })

In [17]:
nasadf.head()

,center,center_search_status,facility,occupied,status,url,record_date,last_update,country,location,city,state,zipcode,latitude,longitude
0,Stennis Space Center,Public,Test Stand A-2 #4122,1964-01-01,Inactive,http://rockettest.nasa.gov,1996-03-01,2015-04-06,US,"Stennis Space Center, MS",Stennis Space Center,MS,39529-6000,30.385948,-89.604486
1,Ames Research Center,Public,N229 - EXPER. AEROTHERMODYNAMIC FAC.: ELECTRIC...,1961-01-01,Active,http://thermo-physics.arc.nasa.gov,1996-03-01,2014-06-13,US,"Moffett Field, CA",Moffett Field,CA,94035,37.41412,-122.052585
2,Goddard Space Flight Center,Public,"500 WSC: Antenna, 10m S-band",1977-01-01,Active,NaN,1996-03-01,2013-08-15,US,"Greenbelt, MD",Greenbelt,MD,20771,38.99538,-76.853161
3,Glenn Research Center,Public,Vacuum Facility 11 (VF-11) Bldg. 16,NaN,Active,http://facilities.grc.nasa.gov/epl/index.html,2010-07-15,2015-03-04,US,"Cleveland, OH",Cleveland,OH,44135,41.430364,-81.808561
4,Marshall Space Flight Center,Public,Materials Testing & Research Facility 4464,NaN,Active,NaN,2007-03-08,2014-03-31,US,"Huntsville, AL",Huntsville,AL,35812,34.729538,-86.585283


# LOAD

In [18]:
# Connect to local database
rds_connection_string = "<username>:<password>@127.0.0.1/space_db"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')

In [20]:
launchlog.to_sql(name='launch_log', con=engine, if_exists='append', index=False)
nasadf.to_sql(name='nasa_facilities', con=engine, if_exists='append', index=False)
spacexdf.to_sql(name='space_x_launches', con=engine, if_exists='append', index=False)

In [21]:
# Confirm data has been added by querying the customer_name table
pd.read_sql_query('select * from launch_log', con=engine).head()

,launch,launch_date_utc,COSPAR,pl_name,orig_pl_name,SATCAT,lv_type,lv_sn,site,suc,Ref,launch_date
0,1957 ALP,1957 Oct 4 1928:34,1957 ALP 2,1-y ISZ,PS-1,S00002,Sputnik 8K71PS,M1-PS,NIIP-5 LC1,S,Energiya,1957-10-04
1,1957 BET,1957 Nov 3 0230:42,1957 BET 1,2-y ISZ,PS-2,S00003,Sputnik 8K71PS,M1-2PS,NIIP-5 LC1,S,Grahn-WWW,1957-11-03
2,1958 ALP,1958 Feb 1 0347:56,1958 ALP,Explorer 1,Explorer 1,S00004,Jupiter C,RS-29 UE,CC LC26A,S,JunoFam,1958-02-01
3,1958 BET,1958 Mar 17 1215:41,1958 BET 2,Vanguard I,Vanguard Test Satellite,S00005,Vanguard,TV-4,CC LC18A,S,SP-4202,1958-03-01
4,1958 GAM,1958 Mar 26 1738:03,1958 GAM,Explorer 3,Explorer 3,S00006,Jupiter C,RS-24 UT,CC LC5,S,JunoFam,1958-03-02


In [22]:
engine.table_names()

['launch_log', 'nasa_facilities', 'space_x_launches']

# QUERY

In [25]:
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')
session = Session(engine)
inspector = inspect(engine)
inspector.get_table_names()

['launch_log', 'nasa_facilities', 'space_x_launches']

In [24]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# We can view all of the classes that automap found
Base.classes.keys()

[]

In [ ]:
stuff = session.query(func.strftime(Table.date)).order_by(Table.date.desc()).first()